In [3]:
import numpy as np

# Variables

Helium viscosity from 2K to 5K: https://royalsocietypublishing.org/doi/10.1098/rspa.1950.0181

Gas helium viscosity: https://pressbooks.uiowa.edu/clonedbook/chapter/viscosity-and-laminar-flow-poiseuilles-law/#import-auto-id3073392

Stainless steel thermal conductivity: https://thermtest.com/thermal-conductivity-of-steel

In [62]:
OD_above_50K = 0.25*0.0254 #m
OD_below_50K = 0.125*0.0254 #m

thickness_above_50K = (0.25-0.18)*0.0254 #m, capillary wall thickness
thickness_below_50K = (0.125-0.085)*0.0254 #m, capillary wall thickness

length_above_50K = 10*0.0254 #m
length_below_50K = 4*20*0.0254 #m

pressure_warm_end_condensation = 1*10**5 #Pa
pressure_cold_end_condensation = 0.25*10**5 #Pa, approximation
pressure_50K_condensation = 0.5*10**5 #Pa, approximation. By ideal gas law: pressure_warm_end*42/300

pressure_warm_end_evaporation = 0.01*10**5 #Pa
pressure_cold_end_evaporation = 0.25*10**5 #Pa, approximation
pressure_50K_evaporation = 0.05*10**5 #Pa, approximation. By ideal gas law: pressure_warm_end*42/300

viscosity_gas = 0.0196*10**(-3) #Pa*s, https://pressbooks.uiowa.edu/clonedbook/chapter/viscosity-and-laminar-flow-poiseuilles-law/#import-auto-id3073392
viscosity_liquid = 0.03*10**(-3) #Pa*s, https://royalsocietypublishing.org/doi/epdf/10.1098/rspa.1950.0181

stainless_steel_thermal_conductivity = 15*10**(-3) #W/K, https://thermtest.com/thermal-conductivity-of-steel

helium_volume = 1 #m^3, approximation

Poiseuille's Law: https://pressbooks.uiowa.edu/clonedbook/chapter/viscosity-and-laminar-flow-poiseuilles-law/

In [13]:
#radius is the inner radius
def getTubeVolmeFlowRate(viscosity, startPressure, endPressure, length, radius):
    R = 8*viscosity*length/(np.pi*(radius**4))
    return (startPressure-endPressure)/R

Fourier's law of heat conduction for cylinders, page 8 on https://www.sfu.ca/~mbahrami/ENSC%20388/Notes/Staedy%20Conduction%20Heat%20Transfer.pdf

In [9]:
def getTubeHeatFlowRate(outerTemperature, innerTemperature, outerRadius, innerRadius, length, thermalConductivity):
    R = (outerRadius - innerRadius)/(4*np.pi*innerRadius*outerRadius*thermalConductivity)
    return (innerTemperature - outerTemperature)/R

# Gas throughput

## Condensation time

In [70]:
inner_radius_above_50K = OD_above_50K - thickness_above_50K
condensation_volume_flow_rate_above_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_warm_end_condensation,pressure_50K_condensation,length_above_50K,inner_radius_above_50K)
condensation_time_above_50K = helium_volume/condensation_volume_flow_rate_above_50K
print(condensation_volume_flow_rate_above_50K)
print(condensation_time_above_50K) #in seconds

1.723316272798174
0.580276537617953


In [71]:
inner_radius_below_50K = OD_below_50K - thickness_below_50K
condensation_volume_flow_rate_below_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_50K_condensation,pressure_cold_end_condensation,length_below_50K,inner_radius_below_50K)
condensation_time_below_50K = helium_volume/condensation_volume_flow_rate_below_50K
print(volume_flow_rate_below_50K)
print(condensation_time_below_50K) #in seconds

0.005355878159813628
186.71074474830803


The condensation time will be limited by the section below the 50K plate, which is 186 seconds.

## Evaporation time

In [68]:
inner_radius_above_50K = OD_above_50K - thickness_above_50K
evaporation_volume_flow_rate_above_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_50K_evaporation,pressure_warm_end_evaporation,length_above_50K,inner_radius_above_50K)
evaporation_time_above_50K = helium_volume/evaporation_volume_flow_rate_above_50K
print(volume_flow_rate_above_50K)
print(evaporation_time_above_50K) #in seconds

1.723316272798174
7.253456720224411


In [72]:
inner_radius_below_50K = OD_below_50K - thickness_below_50K
evaporation_volume_flow_rate_below_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_cold_end_evaporation,pressure_50K_evaporation,length_below_50K,inner_radius_below_50K)
evaporation_time_below_50K = helium_volume/evaporation_volume_flow_rate_below_50K
print(volume_flow_rate_below_50K)
print(evaporation_time_below_50K) #in seconds

0.005355878159813628
233.38843093538503


The evaporation time will be limited by the section below the 50K plate, which is 233 seconds.

# Heat Load

In [74]:
heat_flow_rate_above_50K = getTubeHeatFlowRate(42, 300, OD_above_50K, inner_radius_above_50K, length_above_50K, stainless_steel_thermal_conductivity)
heat_load_above_50K = heat_flow_rate_above_50K*evaporation_time_above_50K
print(heat_load_above_50K) #W

5.7598880629860005


In [77]:
heat_flow_rate_4K_50K = getTubeHeatFlowRate(3, 42, OD_below_50K, inner_radius_below_50K, length_below_50K/4, stainless_steel_thermal_conductivity)
heat_load_4K_50K = heat_flow_rate_4K_50K*evaporation_time_below_50K
print(heat_load_4K_50K) #W

11.575712139822404


In [79]:
heat_flow_rate_still_4K = getTubeHeatFlowRate(0.9, 3, OD_below_50K, inner_radius_below_50K, length_below_50K/4, stainless_steel_thermal_conductivity)
heat_load_still_4K = heat_flow_rate_still_4K*evaporation_time_below_50K
print(heat_load_still_4K)

0.623307576759668


In [80]:
heat_flow_rate_cold_still = getTubeHeatFlowRate(0.08, 0.9, OD_below_50K, inner_radius_below_50K, length_below_50K/4, stainless_steel_thermal_conductivity)
heat_load_cold_still = heat_flow_rate_cold_still*evaporation_time_below_50K
print(heat_load_cold_still)

0.2433867680680608


In [81]:
heat_flow_rate_MXC_cold = getTubeHeatFlowRate(0.02,0.08, OD_below_50K, inner_radius_below_50K, length_below_50K/4, stainless_steel_thermal_conductivity)
heat_load_MXC_cold = heat_flow_rate_MXC_cold*evaporation_time_below_50K
print(heat_load_MXC_cold)

0.017808787907419084


In [82]:
#total heat load in W
print(sum([heat_load_above_50K,heat_load_4K_50K,heat_load_still_4K,heat_load_cold_still,heat_load_MXC_cold]))

18.220103335543552
